### ResNet 50 output sizes of stages

In [2]:
# surpress UserWarnings
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [5]:
from mmdet.models import ResNet
import torch
res50 = ResNet(depth=50)
res50.eval()
inputs =  torch.rand(1, 3, 224, 224)
level_outputs = res50.forward(inputs)
for level_out in level_outputs:
    print(tuple(level_out.shape))

(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 14, 14)
(1, 2048, 7, 7)


### adjust keys in checkpoint weights file

In [1]:
import torch
root_path = "../../../"
checkpoint = torch.load(f'{root_path}checkpoints/faster_rcnn_r50_fpn_1x_voc0712_20220320_192712-54bef0f3.pth')
state_dict = checkpoint['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("backbone.","backbone.resnet.")] = state_dict.pop(key)
checkpoint['state_dict'] = state_dict
torch.save(checkpoint, f'{root_path}checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth')

### Test inference FeedbackNet

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv

root_path = "../../../"
# Specify the path to model config and checkpoint file
config_file = f'{root_path}configs/pascal_voc/faster_rcnn_r50fbadd_fpn_1x_voc0712.py'
checkpoint_file = f'{root_path}checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth'

# build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cpu')
model = init_detector(config_file, checkpoint_file, device='cpu')

In [5]:
# will only load it once
img = mmcv.imread(f'{root_path}demo/demo.jpg')
result = inference_detector(model, img)
# visualize the results in a new window
model.show_result(img, result, out_file=f'{root_path}output/result.jpg')

### Model summary

In [ ]:
# Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#print(model)
from torchsummary import summary
summary(model)#, (1, 3, 224, 224))

In [ ]:
from torchsummary import summary
summary(res50, (1, 3, 224, 224))

### Robustness Benchmark results

In [21]:
import mmcv
def get_results(jobname,postfix="_results.pkl"):
    folder = "evaluation/benchmark/"
    path = folder + "rob-bm-full" + jobname
    return mmcv.load(path + postfix)

#out = mmcv.load(folder+"rob-bm_fb-mod.pkl")
#out_results = mmcv.load(folder+"rob-bm_fb-mod_results.pkl")
#out_tpfp = mmcv.load(folder+"rob-bm-full_fb-mod_tpfp.pkl")
    

In [22]:
for job in ["-2_ff-baseline","_ff-baseline","-2_fb-add","_fb-add","_fb-mod"]:
    res = get_results(job)
    last_sevs = list(res.values())[-1].keys()
    print(f'{job}: {res.keys()} severities of last: {last_sevs} \n')

-2_ff-baseline: dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur']) severities of last: dict_keys([0, 1]) 

_ff-baseline: dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur']) severities of last: dict_keys([0, 1, 2, 3]) 

-2_fb-add: dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur']) severities of last: dict_keys([0, 1, 2, 3]) 

_fb-add: dict_keys(['gaussian_noise', 'shot_noise']) severities of last: dict_keys([0, 1, 2, 3]) 

_fb-mod: dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur']) severities of last: dict_keys([0, 1, 2, 3, 4, 5]) 



In [18]:
fb_mod_res = get_results("_fb-mod")
fb_mod_res['zoom_blur'][5]

[{'ap': 0.42176962},
 {'ap': 0.21062574},
 {'ap': 0.1496132},
 {'ap': 0.16844271},
 {'ap': 0.09090909},
 {'ap': 0.2705015},
 {'ap': 0.33141503},
 {'ap': 0.49906787},
 {'ap': 0.09090909},
 {'ap': 0.13461538},
 {'ap': 0.21941742},
 {'ap': 0.32032588},
 {'ap': 0.20525931},
 {'ap': 0.14242424},
 {'ap': 0.22470236},
 {'ap': 0.09090909},
 {'ap': 0.09090909},
 {'ap': 0.3027891},
 {'ap': 0.25945476},
 {'ap': 0.23072976}]

#### Merge the result dicts

In [19]:
#Example
dict_1 = {'John': 15, 'Rick': 10, 'Misa' : 12 }
dict_2 = {'Bonnie': 18,'Rick': 20,'Matt' : 16 }
dict_1.update(dict_2)
print('Updated dictionary:')
print(dict_1)

Updated dictionary:
{'John': 15, 'Rick': 20, 'Misa': 12, 'Bonnie': 18, 'Matt': 16}


#### Process true/false positive lists

In [24]:
out_tpfp = get_results("_fb-mod","_tpfp.pkl")
tp = out_tpfp['shot_noise'][5][12]['tp']
tp_new = [(ind,item) for (ind,item) in enumerate(tp) if item.shape!=(1,0)]
len(tp_new)

158

-----------
format TP-FP list:

| 15  |  5   |  20 | 2 | 4952 | n |
|---|---|---|---|-|-|
|corruptions |severities | classes | tp, fp | images |detections|

-----------
format output of detections:

| 4952  |  20    |    n     |   5   |
|---|---|---|---|
|images |classes | detected bboxes | bbox + certainty |
                     

In [ ]:
""" 20 classes of PASCAL VOC
 0: aeroplane
 1: bicycle
 2: bird
 3: boat
 4: bottle
 5: bus
 6: car
 7: cat
 8: chair
 9: cow
10: diningtable
11: dog
12: horse
13: motorbike
14: person
15: pottedplant
16: sheep
17: sofa
18: train
19: tvmonitor
"""